In [1]:
import numpy as np
import pandas as pd
# neither year nor len are statistically significant predictors of sex
from pathlib import Path
import numpy as np
import pandas as pd

In [2]:
CWD = Path('.').absolute().resolve()
DATA_DIR = '.nlpia2-data'
DATA_FILE = 'baby-names-region.csv.gz'
CWD

PosixPath('/home/hobs/code/tangibleai/nlpia2/src/nlpia2/ch05')

In [3]:
parent = CWD
data_dir = parent / DATA_DIR 
filepath = data_dir / DATA_FILE
for i in range(10):
    print(filepath)
    if filepath.is_file():
        break
    parent = parent.parent
    data_dir = parent / DATA_DIR 
    filepath = data_dir / DATA_FILE
filepath

/home/hobs/code/tangibleai/nlpia2/src/nlpia2/ch05/.nlpia2-data/baby-names-region.csv.gz
/home/hobs/code/tangibleai/nlpia2/src/nlpia2/.nlpia2-data/baby-names-region.csv.gz
/home/hobs/code/tangibleai/nlpia2/src/.nlpia2-data/baby-names-region.csv.gz
/home/hobs/code/tangibleai/nlpia2/.nlpia2-data/baby-names-region.csv.gz


PosixPath('/home/hobs/code/tangibleai/nlpia2/.nlpia2-data/baby-names-region.csv.gz')

In [4]:
df = pd.read_csv(filepath)

In [5]:
np.random.seed(451)
df = df.sample(10_000)
df.head()

,region,sex,year,name,count,freq
6139665,WV,F,1987,Brittani,10,0.000003
2565339,MD,F,1954,Ida,18,0.000005
22297,AK,M,1988,Maxwell,5,0.000001
5114650,TN,F,1972,Charlene,24,0.000008
2126395,KS,M,1954,Todd,11,0.000003


In [6]:
names = df['name'].unique()
names[:10]

array(['Brittani', 'Ida', 'Maxwell', 'Charlene', 'Todd', 'Aubrey',
       'Arianna', 'Otis', 'Trenton', 'Faustino'], dtype=object)

In [7]:
len(names) / len(df)

0.4025

In [8]:
# df = pd.get_dummies(df, columns=['region'])
# df.head()

In [9]:
df = df.groupby(['name', 'sex']).sum()
df.head()

,,year,count,freq
name,sex,,,
Aaden,M,2008,51,0.000015
Aahana,F,2018,26,0.000009
Aahil,M,2019,5,0.000002
Aaleyah,F,2010,17,0.000005
Aalia,F,4033,13,0.000004


In [10]:
df['name'] = df.index.get_level_values('name')
df['sex'] = df.index.get_level_values('sex')
df.head()

,,year,count,freq,name,sex
name,sex,,,,,
Aaden,M,2008,51,0.000015,Aaden,M
Aahana,F,2018,26,0.000009,Aahana,F
Aahil,M,2019,5,0.000002,Aahil,M
Aaleyah,F,2010,17,0.000005,Aaleyah,F
Aalia,F,4033,13,0.000004,Aalia,F


In [11]:
df.query('name == "Chris"')


year  count      freq   name sex
name  sex                                  
Chris F    1983      5  0.000002  Chris   F
      M    7850    239  0.000069  Chris   M

In [12]:
df.loc[pd.IndexSlice['Chris', :]]

,year,count,freq,name,sex
sex,,,,,
F,1983,5,0.000002,Chris,F
M,7850,239,0.000069,Chris,M


In [13]:
df['istrain'] = np.random.rand(len(df)) < .9
df.head()

,,year,count,freq,name,sex,istrain
name,sex,,,,,,
Aaden,M,2008,51,0.000015,Aaden,M,True
Aahana,F,2018,26,0.000009,Aahana,F,True
Aahil,M,2019,5,0.000002,Aahil,M,True
Aaleyah,F,2010,17,0.000005,Aaleyah,F,True
Aalia,F,4033,13,0.000004,Aalia,F,True


In [14]:
istrain = df['istrain']
del df['istrain']
istrain.sum() / len(istrain)

0.9042000943841435

In [15]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(class_weight='balanced')
model

LogisticRegression(class_weight='balanced')

In [16]:
# df['name'] = df.index.values
df['len'] = df['name'].str.len()
X = df[['len']]
y = df['sex']
X.head()

,,len
name,sex,
Aaden,M,5
Aahana,F,6
Aahil,M,5
Aaleyah,F,7
Aalia,F,5


In [17]:
model.fit(X[istrain], y[istrain], sample_weight=df['count'][istrain])

LogisticRegression(class_weight='balanced')

In [18]:
model.score(X[istrain], y[istrain], sample_weight=df['count'][istrain])

0.5764595550189721

In [19]:
model.score(X[~istrain], y[~istrain], sample_weight=df['count'][~istrain])

0.4489395408223583

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(1, 3), lowercase=False)
vectorizer

TfidfVectorizer(analyzer='char', lowercase=False, ngram_range=(1, 3))

In [21]:
vectorizer.fit(df['name'][istrain])

TfidfVectorizer(analyzer='char', lowercase=False, ngram_range=(1, 3))

In [22]:
vecs = vectorizer.transform(df['name'])
vecs = pd.DataFrame.sparse.from_spmatrix(vecs)
vecs.head()


,0,1,2,3,4,5,6,7,8,9,...,3653,3654,3655,3656,3657,3658,3659,3660,3661,3662
0,0.193687,0.395438,0.506137,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.182726,0.373059,0.000000,0.454870,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.185518,0.378759,0.000000,0.461821,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.166921,0.340790,0.000000,0.000000,0.389490,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.214766,0.438473,0.000000,0.000000,0.501131,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
vecs.columns = vectorizer.get_feature_names_out()
vecs.index = df.index
vecs.head()[vecs.columns[:5]]

,,A,Aa,Aad,Aah,Aal
name,sex,,,,,
Aaden,M,0.193687,0.395438,0.506137,0.000000,0.000000
Aahana,F,0.182726,0.373059,0.000000,0.454870,0.000000
Aahil,M,0.185518,0.378759,0.000000,0.461821,0.000000
Aaleyah,F,0.166921,0.340790,0.000000,0.000000,0.389490
Aalia,F,0.214766,0.438473,0.000000,0.000000,0.501131


In [24]:
model = LogisticRegression(class_weight='balanced', max_iter=1000)
model.fit(vecs[istrain], y[istrain], sample_weight=df['count'][istrain])

LogisticRegression(class_weight='balanced', max_iter=1000)

In [25]:
model.score(vecs[~istrain], y[~istrain], sample_weight=df['count'][~istrain])

0.7878135861726527

In [27]:
model.classes_


array(['F', 'M'], dtype=object)

In [27]:
names = ['Kemal', 'Copeland', 'Hobson', 'Vish', 'Vishvesh']
vecs = vectorizer.transform(names)
vecs = pd.DataFrame.sparse.from_spmatrix(vecs)
vecs.columns = vectorizer.get_feature_names_out()
vecs.index = list(zip(names, 'M'*len(names)))
pd.DataFrame(model.predict_proba(vecs)[:,0], index=vecs.index)

,0
"(Kemal, M)",0.024054
"(Copeland, M)",0.311341
"(Hobson, M)",0.022012
"(Vish, M)",0.983591
"(Vishvesh, M)",0.885610


In [29]:
names = ['Maria', 'Syndee', 'Aditi', 'Connstance', 'Connie']
vecs = vectorizer.transform(names)
vecs = pd.DataFrame.sparse.from_spmatrix(vecs)
vecs.columns = vectorizer.get_feature_names_out()
vecs.index = list(zip(names, 'F'*len(names)))
pd.DataFrame(model.predict_proba(vecs)[:,0], index=vecs.index)

,0
"(Maria, F)",0.999952
"(Syndee, F)",0.990495
"(Aditi, F)",0.968280
"(Connstance, F)",0.889784
"(Connie, F)",0.902063
